In [1]:
with open('D:/Users/ihera/OneDrive/Escritorio/test.txt', 'r') as archivo:
    contenido = archivo.read()

print(contenido)

Oh, goddamn
My pain fits in the palm of your freezing hand
Taking mine, but it's been promised to another
Oh, I can't
Stop you putting roots in my dreamland
My house of stone, your ivy grows
And now I'm covered in you


In [10]:
import random
import time
import numpy as np
import heapq
import json
import hashlib

### Fuente de información

In [3]:
class FuenteInformacion:
    def __init__(self, archivo):
        self.archivo = archivo

    def generar_datos(self):
        return self.archivo

In [12]:
def hash_valor(valor):
    hash_object = hashlib.sha256()
    hash_object.update(str(valor).encode())
    return hash_object.hexdigest()

### Transmisor

In [13]:
class Transmisor:
    def __init__(self, fuente, canal, canales):
        self.fuente = fuente
        self.canal = canal
        self.canales = canales
        self.canal_actual = random.choice(self.canales)
        self.codificador = Huffman()

    def transmitir(self):
        paquetes = self.preparar_datos()
        diccionario, codigo = self.elegir_esquema_codificacion(paquetes)
        # TERCER PARCIAL - HASHEO
        hashes = []
        for elemento in codigo:
            hash_elemento = hash_valor(elemento)
            hashes.append(hash_elemento)
        ###
        self.enviar_paquetes(diccionario, hashes)

    def preparar_datos(self):
        datos = self.fuente.generar_datos()
        datos_bytes = datos.encode('utf-8')
        datos_bin = ''.join(format(byte, '08b') for byte in datos_bytes)
        paquetes = [datos_bin[i:i+8] for i in range(0, len(datos_bin), 8)]
        return paquetes

    def elegir_esquema_codificacion(self, paquetes):
        arbol = self.codificador.codigoHuffman(paquetes)
        diccionario = self.codificador.guardarNodos(arbol)
        codigo = self.codificador.codificar(paquetes, diccionario)
        return diccionario, codigo

    def enviar_paquetes(self, diccionario, codigo):
        paquete_actual = 0
        # Transmitir el diccionario primero como un paquete
        diccionario_str = json.dumps(diccionario)
        diccionario_packet = Paquete(diccionario_str, paquete_actual)
        if not self.canal.enviar(diccionario_packet):
            print("Diccionario perdido. Cambiando a canal de respaldo...")
            index = self.canales.index(self.canal_actual)
            self.canal_actual = self.canales[(index + 1) % len(self.canales)]
            print(f'Enviando por el canal: {self.canal_actual}')

        while paquete_actual < len(codigo):
            paquete = Paquete(codigo[paquete_actual], paquete_actual)
            if not self.canal.enviar(paquete):
                index = self.canales.index(self.canal_actual)
                self.canal_actual = self.canales[(index + 1) % len(self.canales)]
                print(f'Paquete perdido.\nCambiando a canal {self.canal_actual}...')
            else:
                paquete_actual += 1


### Canal de comunicación

In [5]:
class CanalComunicacion:
    def __init__(self, velocidad, receptor):
        self.velocidad = velocidad
        self.receptor = receptor
    
    def enviar(self, paquete):
        time.sleep(self.velocidad)
        if random.random() > 0.09:
            if self.receptor.recibir(paquete):
                return True
        return False

### Receptor

In [16]:
class Receptor:
    def __init__(self):
        self.paquetes = []
        self.coding_dict = None
        self.paquete_esperado = 0

    def recibir(self, paquete):
        if paquete.num_paquete == self.paquete_esperado:
            if self.coding_dict is None:
                # El primer paquete contiene el diccionario
                try:
                    coding_dict_str = paquete.datos
                    self.coding_dict = json.loads(coding_dict_str)
                    # TERCER PARCIAL - HASHEO
                    self.coding_dict = {key: hash_valor(valor) for key, valor in self.coding_dict.items()}
                    ### popipo
                except json.JSONDecodeError:
                    pass
            else:
                # Paquetes
                self.paquetes.append(paquete.datos)
                self.paquete_esperado += 1
                return True
        return False
    
    def buscar_simbolo(self, cadena):
        # Convertir el diccionario a una lista de tuplas (código, símbolo)
        sorted_items = sorted(self.coding_dict.items(), key=lambda x: x[1])
        valores_ordenados = [item[1] for item in sorted_items]
        
        izquierda, derecha = 0, len(valores_ordenados) - 1

        while izquierda <= derecha:
            medio = (izquierda + derecha) // 2

            if valores_ordenados[medio] == cadena:
                return sorted_items[medio][0]  # Devolver el símbolo correspondiente al código
            elif valores_ordenados[medio] < cadena:
                izquierda = medio + 1
            else:
                derecha = medio - 1
            
        return None  # Devolver None si no se encuentra el código

    def decodificar_mensaje(self):
        mensaje_binario = ''
        simbolo_actual = ''

        for bit in self.paquetes:
            simbolo_actual += bit
            simbolo = self.buscar_simbolo(simbolo_actual)
            if simbolo is not None:
                mensaje_binario += simbolo
                simbolo_actual = ''

        mensaje_bytes = [int(mensaje_binario[i:i+8], 2) for i in range(0, len(mensaje_binario), 8)]
        mensaje_decodificado = bytes(mensaje_bytes).decode('utf-8')
        return mensaje_decodificado

### Empaquetamiento

In [7]:
class Paquete:
    def __init__(self, datos, num_paquete):
        self.datos = datos
        self.num_paquete = num_paquete

### Esquemas de codificación

In [8]:
class Nodo:
    def __init__(self, sim, freq, left=None, right=None):
        self.freq = freq
        self.sim = sim
        self.left = left
        self.right = right
        self.huff = ''

    #  Determina qué elemento tiene la mayor prioridad y, por lo tanto, se extrae primero
    def __lt__(self, siguiente):
        return self.freq < siguiente.freq

class Huffman:
    def codigoHuffman(self, lista):
        freq = {}
        for elemento in lista:
            if elemento in freq:
                freq[elemento] += 1
            else:
                freq[elemento] = 1

        # Crear un nodo hoja para cada símbolo, asociando un peso según su frecuencia de aparición e insertarlo en la lista ordenada ascendentemente.
        nodos = [Nodo(simbolo, frecuencia) for simbolo, frecuencia in freq.items()]
        heapq.heapify(nodos)

        # Mientras haya mas de un nodo en la lista.
        while len(nodos) > 1:
            # Eliminar los dos nodos con menos probabilidad de la lista
            nodo_izquierdo = heapq.heappop(nodos)
            nodo_derecho = heapq.heappop(nodos)
            # Asignar valor direccional
            nodo_izquierdo.huff = 0
            nodo_derecho.huff = 1
            # Crear un nuevo nodo interno que enlace a los nodos anteriores, asignándoles como peso la suma de los pesos de los nodos hijos.
            nuevo_nodo = Nodo(None, nodo_izquierdo.freq + nodo_derecho.freq)
            nuevo_nodo.left = nodo_izquierdo
            nuevo_nodo.right = nodo_derecho
            heapq.heappush(nodos, nuevo_nodo)

        return nodos[0]
        
    def guardarNodos(self, nodo, val='', huffDict=None):
        if huffDict is None:
            huffDict = {}

        nuevoVal = val + str(nodo.huff)

        if nodo.left:
            self.guardarNodos(nodo.left, nuevoVal, huffDict)
        if nodo.right:
            self.guardarNodos(nodo.right, nuevoVal, huffDict)

        if not nodo.left and not nodo.right:
            huffDict[nodo.sim] = nuevoVal
            
        return huffDict
    
    def codificar(self, cadena, dict):
        codigo = []
        for caracter in cadena:
            codigo.append(dict[caracter])
        return codigo

In [17]:
canales = [1, 2, 3, 4, 5]

fuente_info = FuenteInformacion(contenido)
receptor = Receptor()
canal = CanalComunicacion(0.1, receptor)
transmisor = Transmisor(fuente_info, canal, canales)

transmisor.transmitir()

# Al final de la transmisión, decodificar el mensaje
mensaje_decodificado = receptor.decodificar_mensaje()
print(f"Mensaje decodificado: \n{mensaje_decodificado}")

Diccionario perdido. Cambiando a canal de respaldo...
Enviando por el canal: 5
Paquete perdido.
Cambiando a canal 1...
Paquete perdido.
Cambiando a canal 2...
Paquete perdido.
Cambiando a canal 3...
Paquete perdido.
Cambiando a canal 4...
Paquete perdido.
Cambiando a canal 5...
Paquete perdido.
Cambiando a canal 1...
Paquete perdido.
Cambiando a canal 2...
Paquete perdido.
Cambiando a canal 3...
Paquete perdido.
Cambiando a canal 4...
Paquete perdido.
Cambiando a canal 5...
Paquete perdido.
Cambiando a canal 1...
Paquete perdido.
Cambiando a canal 2...
Paquete perdido.
Cambiando a canal 3...
Paquete perdido.
Cambiando a canal 4...
Paquete perdido.
Cambiando a canal 5...
Paquete perdido.
Cambiando a canal 1...
Paquete perdido.
Cambiando a canal 2...
Paquete perdido.
Cambiando a canal 3...
Paquete perdido.
Cambiando a canal 4...
Paquete perdido.
Cambiando a canal 5...
Paquete perdido.
Cambiando a canal 1...
Paquete perdido.
Cambiando a canal 2...
Paquete perdido.
Cambiando a canal 3...
P